# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686502


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:27,  1.03it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:03,  2.35s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.74s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:32,  1.29s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.02it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  2.53it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  2.87it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  2.46it/s]

2022_mayaw                            0.003241
shirogane                             0.000731
tanya_p                               0.000858
emilineglade                          0.009927
Dynamatt                              0.000608
not-logged-in-975ec4b89e5ce0be836b    0.000957
Staceyann116                          0.002030
not-logged-in-f7e2260ed4a9517a3e4a    0.002821
glorell                               0.025923
not-logged-in-4d75098d04b85e5c847c    0.002428
not-logged-in-4ec54f98b3c226b9b959    0.076182
not-logged-in-9454a205d149d7907cd0    0.003966
not-logged-in-dddbb97cfd84354416f5    0.001310
pangeli5                              0.001579
kc_                                   0.002272
rhondacard                            0.002076
Lavadude                              0.004164
not-logged-in-31ad73e71ce027f8e855    0.002072
hithereeveryone                       0.001972
not-logged-in-09d8614c5eebb3f9b5af    0.002180
lgonzalez62                           0.001757
Cosmos1      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())